# Choosing the Right Tool: A Comprehensive Guide to Reading Microscopy Data in Python

Microscopy data analysis in Python offers a multitude of options, each with its unique strengths and capabilities. Navigating the expansive landscape of tools can be daunting, but selecting the right one can significantly impact your workflow efficiency and data insights. This guide aims to provide a comprehensive overview of the various Python libraries and frameworks available for reading microscopy data. Whether you prioritize speed, versatility, or integration with other data analysis tools, this guide will help you make informed decisions to meet your specific requirements. Explore the possibilities and empower your microscopy data analysis with the right tools for the job.

## Import and Data Path Assignment

To begin our analysis, we first need to import the necessary libraries and assign the path to our data files. This step ensures that we have access to the tools and resources required for the subsequent tasks.

Let's get started by executing the following code:

In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler
%load_ext memory_profiler

import matplotlib.pyplot as plt
import skimage.io
import numpy as np
import tifffile

import nima_io.read as ir

from pathlib import Path

tdata = Path("../../tests/data/")

# lif = tdata / "2015Aug28_TransHXB2_50min+DMSO.lif"
img_tile = tdata / "t4_1.tif"  # C=3 T=4 S=15
img_void_tile = tdata / "tile6_1.tif"  # C=4 T=3 S=14 scattered
# imgsingle = tdata / "exp2_2.tif"  # C=2 T=81

# mcts = tdata / "multi-channel-time-series.ome.tif"  # C=3 T=7
# bigtiff = tdata / "LC26GFP_1.tf8"  # bigtiff

# slif = str(lif)
simg_tile = str(img_tile)
simg_void_tile = str(img_void_tile)
# simgsingle = str(imgsingle)
# smcts = str(mcts)
# sbigtiff = str(bigtiff)

So, we have few options to open microscopy datafiles:

- skimage.io.imread
- skimage.io.imread_collection
- tifffile.TiffFile
- tifffile.TiffSequence
- bioformats.ImageReader

Imagej hyperstack are organized as **TZCYXS**.

Holoview can also be used. Check availability of reading from disk a la memmap.

Bioformats claims the following standard: separate tiff for each channel and for each time point.
Thus a ome.tif would contain a single plane or a zstack. 

What about tiles? Bioformats has some 6D, 7D and 8D working around 5D. But how is exactly defined 5D?

## Skimage and Tifffile

In [ ]:
t1 = skimage.io.imread(img_tile, plugin="tifffile")
t2 = skimage.io.imread(img_void_tile, plugin="tifffile")
t1.shape, t2.shape

In [ ]:
tf1 = tifffile.imread(img_tile)
tf2 = tifffile.imread(img_void_tile)
tf1.shape, tf2.shape

Only for tiff data files provides:

- sequence
- OME metadata
- memmap
- zarr

In [ ]:
fp1glob = str(tdata / "im1s1z3c5t_?.ome.tif")

tifs = tifffile.TiffSequence(fp1glob)
d = tifs.asarray()
print(d.shape)
print(tifs.shape)

In [ ]:
with tifffile.TiffFile(img_tile) as tif:
    tag = tif.pages[0].tags["ImageDescription"]

tag.value

## nima_io
### read

In [ ]:
r = ir.read(simg_void_tile)
r

### read2

In [ ]:
r2 = ir.read2(simg_void_tile)
r2[0]

### Stitch

In [ ]:
f = ir.stitch(r[0], r[1], c=2, t=2)
skimage.io.imshow(f)

In [ ]:
r[0]["series"][2]["PositionXYZ"]

## nima_io.read

| function   |  time | Pos in md | extra md | no errors | note                     |
|------------|-------|-----------|----------|-----------|--------------------------|
| read       |  4.68 | yes       | no       | no        |                          |
| read2      |  12.4 | no        | yes      | no        |                          |
| read_wrap  |     - | yes       | no       | yes       | It calls read            |
|----
| read_pims  |  2.57 | yes       | no       | yes       | extra pims DIMS          |
| read_jpype | 0.263 | yes       | yes      | yes       |                          |

Summary:

- The Jpype approach stands out for its thoroughness and performance.
- Scyjava, built on Jpype, is now superseding it.
- Functions like read_inf and read_bf may be candidates for removal from the convenient library.

In the following, various bioformats implementations are explored. While python-bioformats produces a verbose warning log through python-javabridge, the numerous metadata linked to individual planes are not returned by the external library. Hence, I decided to develop a small library to handle additional (neglected) metadata, such as acquisition stage position (essential for reconstructing tiled images) and illumination and emission settings.

### read_jpype

In [ ]:
rjpype = ir.read_jpype(simg_void_tile)

In [ ]:
(core_md_jpype, (wr_jpype, type, extra_md_jpype)) = rjpype

In [ ]:
# read_dictionary = np.load("extra_md_read2.npy", allow_pickle=True).item()

sorted_dict = {k: r2[0][k] for k in sorted(r2[0].keys())}
sorted_dictJ = {k: extra_md_jpype[k] for k in sorted(extra_md_jpype.keys())}

In [ ]:
sorted(extra_md_jpype.keys()) == sorted(r2[0].keys())

In [ ]:
r2[0].keys() - extra_md_jpype.keys()

In [ ]:
r2[0]["Creator"]

In [ ]:
# r2[0].pop("Format")
r2[0].pop("Creator")

In [ ]:
sorted_dict = {k: r2[0][k] for k in sorted(r2[0].keys())}
for k in sorted_dict:
    if not sorted_dict[k] == sorted_dictJ.get(k):
        print(k)

In [ ]:
r2[0]["LightPathExcitationFilterRefCount"]

In [ ]:
print(
    r2[0]["ChannelLightSourceSettingsAttenuation"],
    extra_md_jpype["ChannelLightSourceSettingsAttenuation"],
)
print(r2[0]["PixelsBigEndian"], extra_md_jpype["PixelsBigEndian"])
print(r2[0]["PixelsInterleaved"], extra_md_jpype["PixelsInterleaved"])

In [ ]:
f = ir.stitch(core_md_jpype, wr_jpype, c=1, t=0)
skimage.io.imshow(f)

In [ ]:
core_md_jpype["series"][0]["PositionXYZ"]

### PIMS

Which is currently unable to download loci_tools.jar.

**I really like the frame metadata t_s, x_um, y_um and z_um.
Every array (2D, 3D, ..., n-D) having those metadata in common are contained in the Frame obj: a numpy array with metadata(dict) and frame_no(int).**

Are fs.bundle_axes (fs.frame_shape), fs.iter_axes and fs.default_coords overcomplicated?

Anyway: iter=0 == iter=n which is at least unexpected.

In [ ]:
rpims = ir.read_pims(img_void_tile)
rpims

In [ ]:
import scyjava

scyjava.jvm_started()

In [ ]:
md, wr = ir.read(simg_tile)

In [ ]:
mdata = wr.rdr.getMetadataStore()

In [ ]:
root = mdata.getRoot()

In [ ]:
im0 = root.getImage(0)

In [ ]:
pixels = im0.getPixels()

In [ ]:
for idx in range(pixels.sizeOfTiffDataList()):
    tiffData = pixels.getTiffData(idx)
    c = tiffData.getFirstC().getValue().intValue()
    t = tiffData.getFirstT().getValue().intValue()
    print("TiffData: c=%d, t=%d" % (c, t))

## ImageIO

In [ ]:
from imageio.v3 import imread

%timeit imread(img_void_tile, index=13)
%memit imread(img_void_tile, index=13)
i = imread(img_void_tile, index=13)
i.shape

In [ ]:
i.nbytes, 512**2 * 3 * 4 * 2

It can read tif (tf8) files. Series might be passed using `index` (you need to know in advance).

## AICSImageIO

In [ ]:
from aicsimageio import AICSImage

i = AICSImage(img_void_tile)
# i = AICSImage(img_void_tile, reconstruct_mosaic=True)
# i_lif = AICSImage(lif)

In [ ]:
i.ome_metadata

In [ ]:
i.metadata.images[0].pixels.channels[0].light_source_settings.attenuation

In [ ]:
i.scenes

In [ ]:
i.get_dask_stack()

Mosaic stitch is not supported on tif files; so I will use my function relying on the PositionXYZ metadata.

## dask_image

In [ ]:
from dask_image.imread import imread

i = imread(img_void_tile)

In [ ]:
i

In [ ]:
imread(lif)

Somehow it uses bioformats and can handle lif. No mosaic, no metadata though.

**Pycroscopy** https://pypi.org/project/pycroscopy/ is not reading lif nor ome-tif at the moment.

**large-image[all]** failed to install.

**pyimagej** need conda?

## bioio-bioformats

In [ ]:
import bioio_ome_tiled_tiff

In [ ]:
bioio_ome_tiled_tiff.Reader(str(img_tile))

In [ ]:
import bioio_bioformats

im = bioio_bioformats.Reader(img_void_tile)
bioio_bioformats.ReaderMetadata(img_void_tile)

In [ ]:
im.xarray_dask_data

In [ ]:
i = bioio_bioformats.Reader(img_tile)
i.data.shape

In [ ]:
i.xarray_dask_data.attrs["processed"]

In [ ]:
import nima_io.read as ir

In [ ]:
ir.bioformats.get_omexml_metadata(img_tile)

In [ ]:
unp = i.xarray_dask_data.attrs["unprocessed"]
?ir.get_md_dict

In [ ]:
i.ome_metadata

In [ ]:
stk = i.get_dask_stack()

In [ ]:
stk.A

## bfio

In [ ]:
import bfio

bfio.BioReader(img_void_tile)

In [ ]:
rdr = bfio.BioReader(img_void_tile)
%timeit i = rdr.read()
i = rdr.read()
i.shape

In [ ]:
rdr.metadata

In [ ]:
rdr.ps_x

In [ ]:
rdr.close()

## PIMS

In [ ]:
import pims

%timeit fs = pims.Bioformats(img_void_tile)
fs = pims.Bioformats(img_void_tile)
fs.sizes

## PyOMETiff

In [ ]:
import pyometiff

%timeit rdr = pyometiff.OMETIFFReader(fpath=img_void_tile)
rdr = pyometiff.OMETIFFReader(fpath=img_void_tile)

In [ ]:
%timeit r = rdr.read()
res = rdr.read()

In [ ]:
res[2]

In [ ]:
pyometiff.OMETIFFReader._get_metadata_template()

## Final Note

I will keep 

0. Read
1. stitch
2. md_grouping

- impy
- napari.read
- pycromanager
- microscope
- python-microscopy